### Tools

**Initialing a tool**

In [12]:
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two numbers and return the result."""
    return a + b




In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Method 1: @tool decorator
@tool
def calculator(expression: str) -> str:
    """
    Performs mathematical calculations.

    Args:
        expression: A mathematical expression as a string (e.g., "2 + 2", "10 * 5")

    Returns:
        The result of the calculation as a string
    """
    try:
        result = eval(expression)  # Note: Use safe_eval in production!
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def word_counter(text: str) -> str:
    """
    Counts words in a given text.

    Args:
        text: The text to count words in

    Returns:
        Number of words as a string
    """
    word_count = len(text.split())
    return f"Word count: {word_count}"

@tool
def text_reverser(text: str) -> str:
    """
    Reverses the given text.

    Args:
        text: Text to reverse

    Returns:
        Reversed text
    """
    return f"Reversed: {text[::-1]}"

# List of tools
tools = [calculator, word_counter, text_reverser]

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant with access to various tools."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# Testa
result = agent_executor.invoke({
    "input": "Calculate 25 * 17, then count words in 'Hello world from LangChain', then reverse 'Python'"
})

print(f"\n{'='*60}")
print(f"Answer: {result['output']}")
print(f"{'='*60}")


In [5]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Use @tool decorator for simple tools
@tool
def word_lentgth(input: str) -> str:
    """finds the word length"""
    length=len(input.strip())
    return str(length)

tools = [word_lentgth]

# Use ChatPromptTemplate with placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "System message"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),  # Required for tool calling
])

# create_tool_calling_agent is the modern way
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor wraps the agent
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# invoke() is the modern execution method
result = agent_executor.invoke({"input": "Find the length of 'langchain' if length is < 10 answer the question 1 else 2. Qusetions: 1.who is president of russia. 2.who is president of USA"})
print(result['output'])




> Entering new AgentExecutor chain...

Invoking: `word_lentgth` with `{'input': 'langchain'}`


9The length of 'langchain' is 9. Since 9 is less than 10, the answer to question 1 is: The president of Russia is Vladimir Putin.

> Finished chain.
The length of 'langchain' is 9. Since 9 is less than 10, the answer to question 1 is: The president of Russia is Vladimir Putin.
